#**Transformers con Hugging Face**

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 97.4 MB/s eta 0:00:00


**Primer Modelo**

In [ ]:

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
import torch
import torch.nn.functional as F


Sentiment Analysis

In [ ]:
classifier =  pipeline("sentiment-analysis")
res = classifier("We are very happy to use this library")



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
print(res)

[{'label': 'POSITIVE', 'score': 0.9998295307159424}]


Ver otros pipelines: https://huggingface.co/transformers/v3.2.0/main_classes/pipelines.html#

**Segundo Modelo**

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment" #distilbert-base-uncased-finetuned-sst-2-english"
classifier =  pipeline("sentiment-analysis", model=model_name)

In [ ]:
results = classifier(["we are happy","we are sad", "I feel hungry"])
for result in results:
	print(result)

{'label': '5 stars', 'score': 0.5690038800239563}
{'label': '1 star', 'score': 0.3714505136013031}
{'label': '2 stars', 'score': 0.3204992413520813}


EN español?

buscar un modelo aqui: https://huggingface.co/models


In [ ]:
model_name = "finiteautomata/beto-sentiment-analysis"
clasificador =  pipeline("sentiment-analysis", model=model_name)
results = clasificador(["No fue buena la atención","La atención fue más o menos", "cierren mi cuenta"])
for result in results:
	print(result)

{'label': 'NEG', 'score': 0.9947535991668701}
{'label': 'NEU', 'score': 0.9979003667831421}
{'label': 'NEG', 'score': 0.7338845133781433}


**Tercer Modelo**

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier =  pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [ ]:
results = classifier(["we are happy","we are sad"])

for result in results:
	print(result)

{'label': 'POSITIVE', 'score': 0.9998812675476074}
{'label': 'NEGATIVE', 'score': 0.9977351427078247}


Tokenizer

In [ ]:
tokens = tokenizer.tokenize("we are very upset")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tokenizer("we are very upset")

print(f' Tokens: {tokens}')
print(f' Token IDs: {token_ids}')
print(f' Input IDs: {input_ids}')


 Tokens: ['we', 'are', 'very', 'upset']
 Token IDs: [2057, 2024, 2200, 6314]
 Input IDs: {'input_ids': [101, 2057, 2024, 2200, 6314, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}


**Cuarto Modelo**

In [ ]:
X_train = ["we are happy","we are sad"]
batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt") # pt means PyTorch
print(batch)


{'input_ids': tensor([[ 101, 2057, 2024, 3407,  102],
        [ 101, 2057, 2024, 6517,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}


In [ ]:
with torch.no_grad():
	outputs = model(**batch, labels=torch.tensor([1,0]))
	print(outputs)
	predictions = F.softmax(outputs.logits, dim=1)
	print(predictions)
	labels = torch.argmax(predictions, dim=1)
	print(labels)
	labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
	print(labels)


SequenceClassifierOutput(loss=tensor(0.0012), logits=tensor([[-4.3440,  4.6950],
        [ 3.2917, -2.7962]]), hidden_states=None, attentions=None)
tensor([[1.1868e-04, 9.9988e-01],
        [9.9774e-01, 2.2649e-03]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']


**Guardando un modelo**

In [ ]:
#saving
save_directory = "saved"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)
#carga
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSequenceClassification.from_pretrained(save_directory) #


**Quinto Modelo: Español**

Sentiment analysis

In [ ]:
model_name = "finiteautomata/beto-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)



In [ ]:
X_train_esp = ["No me gusta el banco", "La atención es muy buena", "mas o menos","Bastante mala la atención"]
batch = tokenizer(X_train_esp, padding=True, truncation=True, max_length=512, return_tensors="pt") # pt means PyTorch


In [ ]:
with torch.no_grad():
	outputs =  model(**batch)
	label_ids =  torch.argmax(outputs.logits, dim=1)
	print(label_ids)
	labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]
	print(labels)



tensor([0, 2, 1, 0])
['NEG', 'POS', 'NEU', 'NEG']
